<a href="https://colab.research.google.com/github/ch00226855/CMP414765Fall2022/blob/main/Week10_NeuralNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 10
# An Introduction to Neural Networks

**Training large neural networks requires a lot of calculation.** Please turn on GPU computing from "Edit" -> "Notebook Setting" -> "Hardware Acceleration" before running the code below.

# Build a Classifier for Hand-Written Digits

Adapted from [TensorFlow tutorial](https://www.tensorflow.org/tutorials/quickstart/beginner)

1. Build a neural network that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

*Readings: Chapter 10 Introduction to Articial Neural Networks with Keras*

In [ ]:
# import tensorflow
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [ ]:
# Load and prepare the MNIST dataset.
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert the data from integers to floating-point numbers
x_train, x_test = x_train / 255.0, x_test / 255.0

print(x_train.shape, x_test.shape)

In [ ]:
# extract the first image in x_train
idx = 54321
img = x_train[idx, :, :] # : means that we include all indices
print("Shape of the image:", img.shape)
plt.imshow(img, cmap="Greys")

In [ ]:
print("Shape of y_train:", y_train.shape)
y_train[idx]

Now let's build a neural network model.

In [ ]:
# Build a neural network model by stacking layers.
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)), # convert a 28*28 matrix into a 784 1D array
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation="softmax")                                   
])

- The first line creates a `Sequential` model. This is the simplest kind of Keras
model, for neural networks that are just composed of a single stack of layers, connected sequentially. This is called the sequential API.
-Next, we build the first layer and add it to the model. It is a Flatten layer whose
role is simply to convert each input image into a 1D array
- Next we add a Dense hidden layer with 128 neurons. It will use the ReLU activation function.
- Next we add a second Dense hidden layer with 128 neurons, also using the ReLU
activation function.
- Finally, we add a Dense output layer with 10 neurons (one per class), using `softmax` activation function.


In [ ]:
# For each example the model returns a vector of "logits", one for each class.
index = 123
probs = model(x_train[index:(index+1)])
print(probs)

In [ ]:
# The model makes prediction based on the largest probability
class_prediction = np.argmax(probs)
print(class_prediction)

In [ ]:
# Visualize this image
plt.imshow(x_train[index].reshape([28, 28]), cmap=plt.cm.binary)

The prediction accuracy is low, since no training has been performed yet. Let's introduce a function that measures the prediction error.

In [ ]:
# Let's introduce a function that measures the prediction error.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
loss_fn(y_train[index:(index+1)], probs).numpy()

In [ ]:
# Set up the training environment
model.compile(optimizer='sgd',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# The Model.fit method adjusts the model parameters to minimize the loss
history = model.fit(x_train, y_train, epochs=5)

The `fit()` method returns a `History` object containing
- the training parameters (`history.params`), 
- the list of epochs it went through (h`istory.epoch`), 
- a dictionary (`history.history`) containing the loss and extra metrics it
measured at the end of each epoch on the training set and on the validation set

In [ ]:
# The above loss and accuracy is for the training data. Let's evaluate the model on the test set.
model.evaluate(x_test, y_test)

In [ ]:
# Show the confusion matrix
from sklearn.metrics import confusion_matrix
predictions = model.predict(x_test)
classes = np.argmax(predictions, axis=1)
mat = confusion_matrix(y_test, classes)
print(mat)

In [ ]:
# Visualize the training process
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.ylim([0, 1]) # set the vertical range to [0-1]
plt.show()

The image classifier is now trained to ~98% accuracy on this dataset. Let's create a test case ourselves. For example, we can use MS Paint to draw a digit. Remember to resize the canvas to 28*28 pixels

Upload the image to Colab environment by clicking the "Upload to Session Storage" button in the File tab on the left.

In [ ]:
# import pillow for image transformation
import PIL
img = PIL.Image.open("Test.png")
img = img.convert('1') # convert image to black and white
print(img.size)

In [ ]:
# Convert the image to the proper format


In [ ]:
# Obtain predictions from the model


In [ ]:
# y_test_pred = np.argmax(model(x_test).numpy())
y_test_pred = []
raw_predictions = model(x_test).numpy()
for row_idx in range(raw_predictions.shape[0]):
    logits = raw_predictions[row_idx, :]
    probs = tf.nn.softmax(logits).numpy()
    class_pred = np.argmax(probs)
    y_test_pred.append(class_pred)
print(y_test_pred)

In [ ]:
# Save a model
model.save("my_keras_model.h5")

In [ ]:
# Restore a model
model = tf.keras.models.load_model("my_keras_model.h5")

# Exercise 1: Are larger models better?
Modify the neural network model in one of the following ways:
1. Increate the number of neurons from 128 to 256.
2. Add another layer of 128 nodes.
Report the accuracy and the confusion matrix on the test set.



## Exercise 2: Train a Neural Network That Learns the XOR Operation